# Cluster each modality separately and then integrate across the modalities
- cluster RNA with [scVI](https://docs.scvi-tools.org/en/1.3.0/tutorials/notebooks/quick_start/api_overview.html)
- cluster ATAC with [peakVI](https://docs.scvi-tools.org/en/1.3.0/tutorials/notebooks/atac/PeakVI.html)
- integrate RNA and ATAC clusters with [MOSCOT TranslationProblem](https://moscot.readthedocs.io/en/latest/notebooks/tutorials/600_tutorial_translation.html)

In [ ]:
!date

#### import libraries

In [ ]:
import scvi
import scanpy as sc
from anndata import AnnData
from sklearn.metrics import silhouette_score
from numpy import arange, mean
import moscot.plotting as mtp
from moscot.problems.cross_modality import TranslationProblem
from pandas import DataFrame
from scipy import sparse
import scipy
import numpy as np
import torch

import matplotlib.pyplot as plt
from matplotlib.pyplot import rc_context
from seaborn import lineplot

scvi.settings.seed = 42

%matplotlib inline
# for white background of figures (only for docs rendering)
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}
%config InlineBackend.figure_format='retina'

In [ ]:
# make sure we have the cuda jax install and it can see the gpu
import jax
print(f'{jax.devices()}')

In [ ]:
# import importlib
# import moscot.problems.cross_modality
# importlib.reload(moscot.problems.cross_modality)
# from moscot.problems.cross_modality import TranslationProblem

#### set notebook variables

In [ ]:
# variables and constants
DEBUG = True
project = 'aging_phase2'
MAX_MITO_PERCENT = 10
TOP_FEATURES_PERCENT = 0.05
RNA_LATENT_KEY = 'X_scVI'
ATAC_LATENT_KEY = 'X_peakVI'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 4000

# directories
wrk_dir = '/labshare/raph/datasets/adrd_neuro/brain_aging/phase2'
quants_dir = f'{wrk_dir}/quants'

# in files
raw_anndata_file =f'{quants_dir}/{project}.multivi.curated_final.h5ad'

if DEBUG:
    print(f'{raw_anndata_file=}')
    print(f'{device=}')

## functions

In [ ]:
def peek_anndata(adata: AnnData, message: str=None, verbose: bool=False):
    if not message is None and len(message) > 0:
        print(message)
    print(adata)
    if verbose:
        display(adata.obs.head())
        display(adata.var.head())

def foscttm(
    x: np.ndarray,
    y: np.ndarray,
) -> float:
    d = scipy.spatial.distance_matrix(x, y)
    foscttm_x = (d < np.expand_dims(np.diag(d), axis=1)).mean(axis=1)
    foscttm_y = (d < np.expand_dims(np.diag(d), axis=0)).mean(axis=0)
    fracs = []
    for i in range(len(foscttm_x)):
        fracs.append((foscttm_x[i] + foscttm_y[i]) / 2)
    return np.mean(fracs).round(4)

## load the raw multiome data, multiVI anndata

In [ ]:
%%time
adata_multi = sc.read_h5ad(raw_anndata_file)
peek_anndata(adata_multi, 'raw multiVI anndata', DEBUG)
if DEBUG:
    display(adata_multi.obs.modality.value_counts())
    display(adata_multi.var.modality.value_counts())

## split the RNA and ATAC modalities

In [ ]:
%%time
adata_rna = adata_multi[adata_multi.obs.modality.isin(['paired', 'expression']), 
                        adata_multi.var.modality == 'Gene Expression'].copy()
adata_atac = adata_multi[adata_multi.obs.modality.isin(['paired', 'accessibility']), 
                         adata_multi.var.modality == 'Peaks'].copy()
peek_anndata(adata_rna, 'raw RNA anndata', DEBUG)
peek_anndata(adata_atac, 'raw ATAC anndata', DEBUG)
if DEBUG:
    display(adata_rna.obs.modality.value_counts())
    display(adata_rna.var.modality.value_counts())
    display(adata_atac.obs.modality.value_counts())
    display(adata_atac.var.modality.value_counts())    

## cluster the RNA cells

### prep the data, typical preprocessing

In [ ]:
# mitochondrial genes, "MT-" for human, "Mt-" for mouse
adata_rna.var['mt'] = adata_rna.var_names.str.startswith('MT-') 
# ribosomal genes
adata_rna.var['ribo'] = adata_rna.var_names.str.startswith(('RPS', 'RPL'))
# hemoglobin genes
adata_rna.var['hb'] = adata_rna.var_names.str.contains('^HB[^(P)]')
sc.pp.calculate_qc_metrics(adata_rna, qc_vars=['mt', 'ribo', 'hb'], 
                           inplace=True, log1p=True)
# Basic filtering:
adata_rna = adata_rna[adata_rna.obs.pct_counts_mt < MAX_MITO_PERCENT, :]
sc.pp.filter_cells(adata_rna, min_genes=200)
sc.pp.filter_genes(adata_rna, min_cells=3)

n_top_genes = int(adata_rna.n_vars * TOP_FEATURES_PERCENT)
sc.pp.highly_variable_genes(adata_rna, n_top_genes=n_top_genes, 
                            batch_key='gex_pool',flavor='seurat_v3', 
                            subset=True)

peek_anndata(adata_rna, 'prepped RNA anndata', DEBUG)
if DEBUG:
    display(adata_rna.obs.modality.value_counts())

In [ ]:
adata_rna.layers['counts'] = adata_rna.X.copy()  # preserve counts

### setup the RNA anndata for scVI

In [ ]:
scvi.model.SCVI.setup_anndata(
    adata_rna,
    layer="counts",
    categorical_covariate_keys=['gex_pool', 'sample_id'],
    continuous_covariate_keys=['pct_counts_mt', 'pct_counts_ribo'],
)

### create and train the model

In [ ]:
%%time
rna_model = scvi.model.SCVI(adata_rna)
print(rna_model)
display(rna_model.view_anndata_setup())
rna_model.train(batch_size=BATCH_SIZE)

### get and store the model output

In [ ]:
# adata_rna.obsm[RNA_LATENT_KEY] = sparse.csr_matrix(rna_model.get_latent_representation())
adata_rna.obsm[RNA_LATENT_KEY] = rna_model.get_latent_representation()
peek_anndata(adata_rna, 'post latent RNA anndata', DEBUG)
if DEBUG:
    display(adata_rna.obsm[RNA_LATENT_KEY].shape)

### embed the graph based on latent representation

In [ ]:
%%time
sc.pp.neighbors(adata_rna, use_rep=RNA_LATENT_KEY)
sc.tl.umap(adata_rna)
peek_anndata(adata_rna, 'embedded latent RNA anndata', DEBUG)
sc.tl.leiden(adata_rna, key_added='leiden_VI', resolution=0.6, flavor='igraph', n_iterations=2)

### visualize

In [ ]:
with rc_context({'figure.figsize': (8, 8), 'figure.dpi': 100}):
    plt.style.use('seaborn-v0_8-bright')
    sc.pl.umap(adata_rna, color=['leiden_VI'], frameon=False, legend_loc='on data')    

In [ ]:
with rc_context({'figure.figsize': (8, 8), 'figure.dpi': 100}):
    plt.style.use('seaborn-v0_8-bright')
    sc.pl.umap(adata_rna, color=['gex_pool'], frameon=False)        

In [ ]:
with rc_context({'figure.figsize': (8, 8), 'figure.dpi': 100}):
    plt.style.use('seaborn-v0_8-bright')
    sc.pl.umap(adata_rna, color=['cluster_name'], frameon=False, legend_loc='on data')    

In [ ]:
with rc_context({'figure.figsize': (8, 8), 'figure.dpi': 100}):
    plt.style.use('seaborn-v0_8-bright')
    sc.pl.umap(adata_rna, color=['modality'], frameon=False)    

In [ ]:
with rc_context({'figure.figsize': (8, 8), 'figure.dpi': 100}):
    plt.style.use('seaborn-v0_8-bright')
    sc.pl.umap(adata_rna, color=['curated_type'], frameon=False)    

## cluster the ATAC cells

### prep the data, limited ATAC steps

In [ ]:
sc.pp.filter_cells(adata_atac, min_genes=200)
# compute the threshold: 5% of the cells
min_cells = int(adata_atac.n_obs * 0.05)
# in-place filtering of regions
sc.pp.filter_genes(adata_atac, min_cells=min_cells)

n_top_genes = int(adata_atac.n_vars * TOP_FEATURES_PERCENT)
sc.pp.highly_variable_genes(adata_atac, n_top_genes=n_top_genes, 
                            batch_key='atac_pool',flavor='seurat_v3', 
                            subset=True)

peek_anndata(adata_atac, 'prepped ATAC anndata', DEBUG)
if DEBUG:
    display(adata_atac.obs.modality.value_counts())

In [ ]:
adata_atac.layers['counts'] = adata_atac.X.copy()  # preserve counts

### setup the ATAC anndata for scVI

In [ ]:
scvi.model.PEAKVI.setup_anndata(
    adata_atac,
    layer="counts",
    categorical_covariate_keys=['atac_pool', 'sample_id']
)

### create and train the model

In [ ]:
%%time
atac_model = scvi.model.PEAKVI(adata_atac)
print(atac_model)
display(atac_model.view_anndata_setup())
atac_model.train(batch_size=BATCH_SIZE)

In [ ]:
print(atac_model)

### get and store the model output

In [ ]:
# adata_atac.obsm[ATAC_LATENT_KEY] = sparse.csr_matrix(atac_model.get_latent_representation())
adata_atac.obsm[ATAC_LATENT_KEY] = atac_model.get_latent_representation()
peek_anndata(adata_atac, 'post latent ATAC anndata', DEBUG)
if DEBUG:
    display(adata_atac.obsm[ATAC_LATENT_KEY].shape)

### embed the graph based on latent representation

In [ ]:
%%time
sc.pp.neighbors(adata_atac, use_rep=ATAC_LATENT_KEY)
sc.tl.umap(adata_atac)
peek_anndata(adata_atac, 'embedded latent ATAC anndata', DEBUG)
sc.tl.leiden(adata_atac, key_added='leiden_VI', resolution=0.4, flavor='igraph', n_iterations=2)

### visualize

In [ ]:
with rc_context({'figure.figsize': (8, 8), 'figure.dpi': 100}):
    plt.style.use('seaborn-v0_8-bright')
    sc.pl.umap(adata_atac, color=['leiden_VI'], frameon=False, legend_loc='on data')    

In [ ]:
with rc_context({'figure.figsize': (8, 8), 'figure.dpi': 100}):
    plt.style.use('seaborn-v0_8-bright')
    sc.pl.umap(adata_atac, color=['atac_pool'], frameon=False)        

In [ ]:
with rc_context({'figure.figsize': (8, 8), 'figure.dpi': 100}):
    plt.style.use('seaborn-v0_8-bright')
    sc.pl.umap(adata_atac, color=['cluster_name'], frameon=False, legend_loc='on data')    

In [ ]:
with rc_context({'figure.figsize': (8, 8), 'figure.dpi': 100}):
    plt.style.use('seaborn-v0_8-bright')
    sc.pl.umap(adata_atac, color=['modality'], frameon=False)    

In [ ]:
with rc_context({'figure.figsize': (8, 8), 'figure.dpi': 100}):
    plt.style.use('seaborn-v0_8-bright')
    sc.pl.umap(adata_atac, color=['curated_type'], frameon=False)    

## integrate the modalities

### Prepare the TranslationProblem
We need to initialize the TranslationProblem by passing the source and target AnnData objects. After initialization, we need to prepare() the problem. In this particular case, we need to pay attention to 3 parameters:

src_attr: specifies the attribute in AnnData that contains the source distribution. In our case it refers to the key in obsm that stores the ATAC LSI embedding.

tgt_attr: specifies the attribute in AnnData that contains the target distribution. In our case it refers to the key in obsm that stores the RNA PCA embedding.

joint_attr[optional]: specifies a joint attribute over a common feature space to incorporate a linear term into the quadratic optimization problem. Initially, we consider the pure Gromov-Wasserstein setting and subsequently explore the fused problem.

#### it keeps exhausting the GPU resources so try a sample at a time
when trying to run the full dataset keep exhausting the resource on the GPU and changing the batch size doesn't seem to have an effet to try a sample at a time

In [ ]:
adata_atac.obs.sample_id.value_counts().head()

In [ ]:
adata_atac.obs[adata_atac.obs.modality == 'paired'].sample_id.value_counts().head()

In [ ]:
this_sample = 'Aging130'
adata_atac_sub = adata_atac[adata_atac.obs.sample_id == this_sample]
adata_rna_sub = adata_rna[adata_rna.obs.sample_id == this_sample]
print(adata_atac_sub)
print(adata_rna_sub)

In [ ]:
with rc_context({'figure.figsize': (8, 8), 'figure.dpi': 100}):
    plt.style.use('seaborn-v0_8-bright')
    sc.pl.umap(adata_atac_sub, color=['leiden_VI'], frameon=False, 
               legend_loc='on data', title='ATAC leiden_VI') 
    sc.pl.umap(adata_rna_sub, color=['leiden_VI'], frameon=False, 
               legend_loc='on data', title='RNA leiden_VI')     

In [ ]:
# tp = TranslationProblem(adata_src=adata_atac, adata_tgt=adata_rna)
# tp = tp.prepare(src_attr="X_peakVI", tgt_attr="X_scVI", batch_key='sample_id')
tp = TranslationProblem(adata_src=adata_atac_sub, adata_tgt=adata_rna_sub)
tp = tp.prepare(src_attr="X_peakVI", tgt_attr="X_scVI")
# tp = TranslationProblem(adata_src=adata_rna_sub, adata_tgt=adata_atac_sub)
# tp = tp.prepare(src_attr="X_scVI", tgt_attr="X_peakVI")

### Solve the TranslationProblem
In fused quadratic problems, the alpha parameter defines the convex combination between the quadratic and linear terms. By default, alpha = 1, that is, we only consider the quadratic problem, ignoring the joint_attr. We choose a small value for epsilon to obtain a sparse transport map.

In [ ]:
%%time
tp = tp.solve(alpha=1.0, epsilon=1e-3)

### Translate the TranslationProblem
We can now project one domain onto the other. The boolean parameter forward determines the direction of the barycentric projection. In our case, we project the source distribution AnnData (ATAC) onto the target distribution AnnData (RNA), therefore we use forward = True. The function translate() returns the translated object in the target space (or source space respectively).

In [ ]:
translated = tp.translate(source='src', target='tgt', forward=True)

In [ ]:
translated.shape

### Analyzing the translation
We will use the average FOSCTTM metric implemented above to analyze the alignment performance.

In [ ]:
# print(
#     "Average FOSCTTM score of translating ATAC onto RNA: ",
#     foscttm(adata_rna_sub.obsm["X_scVI"], translated),
#     # foscttm(adata_atac_sub.obsm["X_peakVI"], translated),    
# )

In [ ]:
adata = sc.concat(
    [adata_atac_sub, adata_rna_sub],
    join="outer",
    label="batch",
    keys=["ATAC (translated)", "RNA"],
)
adata.obsm["X_translated_1"] = np.concatenate(
    (translated, adata_rna_sub.obsm["X_scVI"]), axis=0
# adata.obsm["X_translated_1"] = np.concatenate(
#     (translated, adata_atac_sub.obsm["X_peakVI"]), axis=0
)
sc.pp.neighbors(adata, use_rep="X_translated_1")
sc.tl.umap(adata)

In [ ]:
with rc_context({'figure.dpi': 100}):
    plt.style.use('seaborn-v0_8-bright')
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    sc.pl.umap(adata, color=["batch"], ax=ax1, show=False)
    ax1.legend()
    ax1.set_title("Colored based on modality")
    sc.pl.umap(adata, color=["cluster_name"], ax=ax2, show=False)
    ax2.set_title("Colored based on cell types")
    plt.tight_layout(pad=3.0)
    plt.show()

In [ ]:
!date